# Функции 2. Генераторы, итераторы, пространства имен

### Ilia Sklonin

In [1]:
!python --version

Python 3.9.18


[EAFP - Easier to Ask for Forgiveness than Permission](https://docs.python.org/3.11/glossary.html#term-EAFP)

### План на сегодня

1. итераторы и итерабельность, модуль `collections.abc`, функция `isinstance`
2. генераторы, инструкция `yield`, выражение для создания генератора
3. пространства имен, `globals()` и `locals()`
4. области видимости, LEGB, `global` и `nonlocal`
5. замыкания, `__closure__`
6. `itertools`, `functools`

Как можно проверить тип объекта? Раньше вы могли делать это таким образом:

In [2]:
if type([1, 2, 3]) == list:
    print("yes, it's list")

yes, it's list


Теперь, если нужно проверить тип переменной, мы будем использовать функцию `isinstance(obj, cls)`. Она позволяет определить, является ли `obj` экземпляром класса `cls`.

In [3]:
isinstance([1, 2, 3], list)

True

In [4]:
isinstance({1: 2}, dict)

True

In [5]:
isinstance(True, str)

False

Очень удобно проверять наличие той или иной функциональности у объекта с помощью модуля `collections.abc` (Abstract Base Classes). Посмотреть список доступных Абстрактных классов можно [здесь](https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes)

In [6]:
from collections.abc import Hashable

In [7]:
isinstance([1, 2, 3], Hashable)

False

При этом вместо одного типа можно передать кортеж типов:

In [8]:
isinstance(True, (bool, str))  # equivalent isinstance(x, A) or isinstance(x, B)

True

In [9]:
isinstance(True, (float, str))

False

In [10]:
isinstance(True, int)

True

In [3]:
# не забудьте заглянуть в help!
help(isinstance)

Help on built-in function isinstance in module builtins:

isinstance(obj, class_or_tuple, /)
    Return whether an object is an instance of a class or of a subclass thereof.
    
    A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
    or ...`` etc.



## Iterable and Iterator

Генераторы и итераторы представляют собой инструменты, которые, как правило, используются для поточной обработки данных. 

Сначала разберемся с итераторами. Итератор — это объект, который позволяет программисту перемещаться по контейнеру, в частности, по спискам. Однако итератор выполняет обход и предоставляет доступ к элементам в контейнере, но не выполняет саму итерацию! Давайте к сути. Есть три понятия, а именно:

- **iterable** (итерабельный) - это свойство, которым обладает тот объект, у которого определен метод `__iter__`, который возвращает итератор. Короче говоря, итерируемый объект — это любой объект, который может предоставить нам итератор.
- **iterator** - объект, у которого определен метод `__next__`. Вот и все.
- **iteration** - это процесс извлечения элемента из чего-либо, например, списка. Когда мы используем цикл для перебора чего-либо, это называется итерацией. Это имя, данное самому процессу.

Прямые определения не очень информативны, поэтому обсудим их на конкретных примерах, а позже поговорим о генераторах.

Проверим, является ли список `iterable` объектом:

In [8]:
values = ['Hello', 'world', '!']
print(values.__iter__())

Чтобы получить итератор, можно вызвать функцию `iter(iterable)`. Синтаксически она выглядит компактнее, чем вызов метода `.__iter__()`

In [11]:
iter(values)

`iterables` могут использоваться в цикле `for` и во многих других местах, где требуется последовательность (`enumerate()`, `zip()`, `map()`, ...). Когда итерируемый объект передается в качестве аргумента встроенной функции `iter()`, она возвращает итератор для объекта. Этот итератор хорош для одного прохода по набору значений. При использовании `iterables` обычно нет необходимости вызывать `iter()` или самостоятельно работать с объектами-итераторами. Цикл `for` делает это автоматически за вас, создавая временную безымянную переменную для хранения итератора на время цикла. [glossary: iterable](https://docs.python.org/3.11/glossary.html#term-iterable)

In [12]:
# не забудьте заглянуть в help(iter)! 
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



Посмотрим на процесс итерации на примере нашего списка `values` и цикла `for`:

In [14]:
for value in values:
    print(value)
    
# Цикл создает временную безымянную переменную, хранящую итератор:
iterator = iter(values)    
for value in iterator:
    print(value)

Hello
world
!
Hello
world
!


А что такое этот итератор? Это объект, представляющий поток(stream) данных. Как было сказано выше, он позволяет получать элементы из последовательности. Посмотрим в переменную `iterator`:

In [15]:
iterator

Это специальный объект перечислитель, который для данной последовательности выдает следующий элемент, либо бросает исключение, если элементов больше нет. Давайте еще раз проитерируемся по нему:

In [16]:
for value in iterator:
    print(value)

Где обещанное исключение? Оно было, но его самостоятельно отловил и обработал цикл `for`.

А почему ничего не напечаталось? Дело в том, что наш итератор уже исчерпал себя: он знает, что отдал все объекты и сейчас находится в конце последовательности.

Чтобы понять этапы итерации подробнее, давайте выполним ее вручную. По определению у итератора должен существовать метод `__next__`. При этом для компактности кода вместо вызова этого метода можно вызывать встроенную функцию `next`.

In [17]:
# пересоздадим итератор
iterator = iter(values)

In [18]:
next(iterator)  # эквивалентно вызову iterator.__next__()

'Hello'

Давайте вызовем `next` еще раз

In [19]:
next(iterator)

'world'

Повторные вызовы метода `__next__()` (или вызов встроенной функции `next()`) возвращают следующие элементы в потоке. 

In [21]:
next(iterator)

StopIteration: 

Если данных больше нет (дошли до конца), возникает исключение `StopIteration`

In [20]:
next(iterator)

StopIteration: 

На этом этапе объект `iterator` исчерпан, и любые дальнейшие попытки получить следующий элемент просто снова выбрасывают `StopIteration`. 

In [21]:
next(iterator)

StopIteration: 

Но мы можем "подвинуть" итератор на нужный нам элемент последовательности, **если** он еще не отдал все свои элементы!

In [4]:
iterator = iter(values)

print(next(iterator))
print(next(iterator))
iterator.__setstate__(0)
print(next(iterator))

Hello
world
Hello


Итераторы должны обладать методом `__iter__()`, который возвращает сам объект итератора, поэтому каждый итератор также является итерабельным и может использоваться там, где принимаются другие итерабельные объекты.

[link to glossary: iterator](https://docs.python.org/3.11/glossary.html#term-iterator)

Саммари:

- iterable - свойство
- iterator - объект
- iteration - процесс

![tit](images/iterable-vs-iterator.png 'title')


Вспомним про модуль `collections.abc` и импортируем оттуда классы `Iterable` и `Iterator`

In [24]:
from collections.abc import Iterable, Iterator

In [23]:
iterable = ['Alice', 'Bob', 'Charlie']
iterator = iter(iterable)

С помощью функции `isinstance` убедимся, что список является итерабельным, а итератор списка является... итератором!

In [29]:
print(iterable)
print(isinstance(iterable, Iterable))

['Alice', 'Bob', 'Charlie']
True


In [30]:
print(iterator)
print(isinstance(iterator, Iterator))

True


Как теперь ответить на вопрос: является ли сам список итератором?

In [25]:
isinstance(iterable, Iterator)

False

## Generator

Генераторы - это итераторы, но вы можете выполнить итерацию по ним только один раз. Это потому, что они не хранят все значения в памяти, они генерируют их "на лету". Вы используете их, перебирая либо с помощью цикла `for`, либо передавая любой функции или конструкции, которая выполняет итерацию.

Проще говоря, теперь вместо хранения всех элементов последовательности в памяти мы будем хранить один объект (генератор), который знает, как сгенерировать каждый элемент. Давайте создадим генератор с помощью уже знакомого нам синтаксиса list comprehension: 

```python
[expression for counter in sequence]
```

, нужно просто вынуть само выражение из квадратных скобок. Но просто так использовать это выражение не получится:

In [32]:
x * x * x for x in range(10)

SyntaxError: invalid syntax (41830029.py, line 1)

Пока что нужно добавить круглые скобки. Мы помогаем интерпретатору понять, что это единое выражение для создания объекта:

In [33]:
(x * x * x for x in range(10))

<generator object <genexpr> at 0x111ab3430>

Создадим для этого объекта переменную `generator_exp` и посмотрим на ее тип:

In [34]:
generator_exp = (x * x for x in range(10))

type(generator_exp)

generator

Мы познакомились с первым способом создания генератора: `generator expression`. Удобно использовать, когда нам нужно получить итератор в каком-то месте кода, но не хочется выделять память для всех его элементов одновременно. Например, нам нужно посчитать сумму элементов длинного списка:

In [ ]:
# не запускать, работает примерно пол минуты
# %%time
# sum([x ** 3 for x in range(10 ** 8)])

результат запуска:
```
CPU times: user 35.3 s, sys: 2.96 s, total: 38.3 s
Wall time: 38.3 s
```

In [ ]:
# не запускать, работает примерно пол минуты
# %%time
# sum(x ** 3 for x in range(10 ** 8))   # generator expression

результат запуска:
```
CPU times: user 34.7 s, sys: 150 ms, total: 34.8 s
Wall time: 34.9 s
```

Многие стандартные библиотечные функции, возвращающие списки в Python 2, были изменены для возврата генераторов в Python 3, поскольку генераторы требуют меньше ресурсов. Давайте напишем генератор, возвращающий по запросу следующее число в двоичной записи:

In [35]:
# функция bin переводит число в двоичную систему счисления
generator = (bin(x) for x in range(16))

for elem in generator:
    print(f'{elem = }')
    if int(elem, base=2) > 7:
        break
        
# аргумент base задает систему счисления, в которой записано число
# т.е. int(number, base=2) переведет number из двоичной системы в десятичную

elem = '0b0'
elem = '0b1'
elem = '0b10'
elem = '0b11'
elem = '0b100'
elem = '0b101'
elem = '0b110'
elem = '0b111'
elem = '0b1000'


Итак:

- **Generator** - особый вид `Iterator`
- **Generator Expression** - способ создания `Generator`.

Следующий способ создания генератора:
- **Generator Function** (в простонародии тоже генератор)

это функция, которая возвращает итератор генератора. Она выглядит как обычная функция, за исключением того, что она содержит [yield statement](https://docs.python.org/3/reference/simple_stmts.html#the-yield-statement) для получения ряда значений, которые можно использовать в цикле `for` или которые могут быть извлечены по одному с помощью функции `next()`. Обычно это понятие относится к функции генератора, но в некоторых случаях может относиться к итератору генератора. В случаях, когда предполагаемое значение неясно, использование полных терминов позволяет избежать двусмысленности.

Ссылки в glossary:
- [generator](https://docs.python.org/3.11/glossary.html#index-19)
- [generator iterator](https://docs.python.org/3.11/glossary.html#term-generator-iterator)
- [generator expression](https://docs.python.org/3.11/glossary.html#index-20)

Кратко: функция-генератор - это функция, в которой присутствует ключевое слово `yield`. При вызове эта функция возвращает генератор!

In [36]:
def generator_function():
    for i in range(10):
        yield i

gen = generator_function()
        
print(f'{gen = }')        

print(f'{type(gen) = }')

for elem in generator_function():
    print(elem)

gen = <generator object generator_function at 0x111ab3820>
type(gen) = <class 'generator'>
0
1
2
3
4
5
6
7
8
9


In [37]:
from collections.abc import Generator 

print(isinstance(generator, Generator))
print(isinstance(generator, Iterator))

True
True


Несколько правил:
- каждый раз, когда внутри функции встречается `yield expression`, генератор останавливается и возвращает объект `expression`
- при следующем запросе генератор продолжает работу с того же места, где он остановился
- `yield` может использоваться несколько раз
- в конце генератор выбрасывает `StopIteration`, как и любой порядочный итератор)

Проверим их с помощью следующего примера:

In [26]:
def generator_func():
    yield 1
    print('эта строка перед циклом')
    for x in range(2, 4):
        yield x
    print('эта строка после цикла')
    yield 10

In [27]:
generator = generator_func()

print(next(generator))

1


In [28]:
print(next(generator))
print(next(generator))


эта строка перед циклом
2
3


In [41]:
print(next(generator))

StopIteration: 

**Note** Если хочется посмотреть на все значения какого-нибудь итератора, то нам надо заставить отдать его все значения. Самый простой способ - передать его в `list`:

In [42]:
list(generator_func())

эта строка перед циклом
эта строка после цикла


[1, 2, 3, 10]

### Примеры генераторов

**№1** Напишите генератор, возвращающий кубы чисел из списка `lst` (гарантируется, что там только числа):

In [43]:
def cubes(lst):
    # your code here
    for element in lst:
        yield element ** 3

In [46]:
# test
cubics = cubes([20, 31, 42])

assert next(cubics) == 8000
assert next(cubics) == 29791
assert next(cubics) == 74088

Генераторы могут быть бесконечными, поэтому:

**№2** Напишите бесконечный $\infty$ генератор кубов целых положительных чисел, начиная с 0 с шагом 2

In [47]:
def inf_cubes():
    # your code here
    i = 0
    while True:
        yield i ** 3
        i += 2

In [48]:
# test
for x in inf_cubes():
    print(x)
    if x >= 1000:
        break

0
8
64
216
512
1000


**№3** Как проверить, является ли объект, создаваемый функцией `range`, итератором:

In [49]:
ran = range(10)
type(ran)

range

In [50]:
next(ran)

TypeError: 'range' object is not an iterator

In [67]:
iter(ran)  # ran.__iter__()

Если вы запутались, кто чем является и как что работает, то попробуйте воспользоваться картинкой с отношениями этих понятий:
- контейнер
- итерабельность
- итератор
- генератор
- функция-генератор
- выражение генератор
- list, dict, set comprehension
- `iter` и `next`


![alt text](images/relationships_gen_iter.png)

## Functional programming

---
_Небольшое замечание, про что этот раздел_

Здесь вы можете погрузиться в определения и целое море примеров про то, что такое функциональное программирование и что питон дает нам для него:
- хабр [Функциональное программирование в Python. Генераторы, как питонячий декларативный стиль](https://habr.com/ru/articles/517438/)
- [Functional Programming HOWTO](https://docs.python.org/3.11/howto/functional.html)
- [Functional Programming Modules](https://docs.python.org/3.11/library/functional.html)

В питоне для ФП есть:
- некоторые встроенные функции: `map`, `zip`, `filter`, `enumerate`
- итераторы и генераторы
- встроенные модули `itertools` и `functools`
- механизм создания функций `lambda`

Мы не будем погружаться в терминологию, лишь посмотрим на некоторые примеры использования указанных выше элементов.

---

Уже известные (может быть еще не все) нам встроенные функции `map`, `zip`, `enumerate`, `filter` как раз и возвращают итераторы!

`map(func, *iterables)` - создает итератор, возвращающий результат работы функции `func`, которой в качестве аргументов приходят элементы последовательностей `*iterables`

In [ ]:
# map(func, *iterables) == map(func, iterables[0], iterables[1], ...)

In [68]:
m = map(str, [1, 2, 3])
list(m)

['1', '2', '3']

Если `func` принимает n аргументов, то и последовательностей должно быть n:

In [69]:
list(map(min, [1, 2, 3], [-1, 5, 42]))  # n = 2

[-1, 2, 3]

In [70]:
list(map(lambda arg1, arg2, arg3: f'{arg1} {arg2} {arg3 ** 3}',  # n = 3
         ['first', 'second', 'third'], 
         ['apple', 'banana', 'strawberry'], 
         [10, 20, 30]))

['first apple 1000', 'second banana 8000', 'third strawberry 27000']

`zip(*iterables)` - возвращает кортежи, собранные из элементов с одинаковым индексом. Например, `zip(iter1, iter2, iter3)` возвращает кортежи из трех элементов по следующему принципу:
- `(iter1[0], iter2[0], iter3[0])`
- `(iter1[1], iter2[1], iter3[1])`
- ...

Число кортежей равно длине самой короткой последовательности:

In [71]:
z = zip([1, 2, 3, 4], 
        'abc', 
        (42, -42))

list(z)

[(1, 'a', 42), (2, 'b', -42)]

Одно из важнейших преимуществ `zip`: повышает читаемость циклов:

In [72]:
first_name = ['Joe','Earnst','Thomas','Martin','Charles']
last_name = ['Schmoe','Ehlmann','Fischer','Walter','Rogan','Green']
age = [23, 65, 11, 36, 83]

for first_name, last_name, age in zip(first_name, last_name, age):
    print(f"{first_name} {last_name} is {age} years old")

Joe Schmoe is 23 years old
Earnst Ehlmann is 65 years old
Thomas Fischer is 11 years old
Martin Walter is 36 years old
Charles Rogan is 83 years old


А вот пример с `unzip` :)

In [73]:
full_name_list = [('Joe', 'Schmoe', 23),
                  ('Earnst', 'Ehlmann', 65),
                  ('Thomas', 'Fischer', 11),
                  ('Martin', 'Walter', 36),
                  ('Charles', 'Rogan', 83)]

first_name, last_name, age = list(zip(*full_name_list))
print(f"{first_name = }\n{last_name = }\n{age = }")

first_name = ('Joe', 'Earnst', 'Thomas', 'Martin', 'Charles')
last_name = ('Schmoe', 'Ehlmann', 'Fischer', 'Walter', 'Rogan')
age = (23, 65, 11, 36, 83)


#### enumerate

`enumerate(iterable, start=0)` - возвращает кортежи вида `(index, element)` из последовательности `iterable`. Если `start != 0`, то `(start + index, element)`

In [74]:
e = enumerate(['aaa', 'bbb', 'ccc'], start=42)
list(e)

[(42, 'aaa'), (43, 'bbb'), (44, 'ccc')]

Как и в случае с `zip`, удобно использовать в циклах:

In [75]:
my_list = ['apple', 'banana', 'grapes', 'pear']
for c, value in enumerate(my_list, 1):
    print(c, value)

1 apple
2 banana
3 grapes
4 pear


In [76]:
for i, value in enumerate(full_name_list):
    if not i % 2:  # напечатаем только четных людей
        print(value)

('Joe', 'Schmoe', 23)
('Thomas', 'Fischer', 11)
('Charles', 'Rogan', 83)


Для фильтрации значений по какому-то условию удобно использовать функцию `filter(function, iterable)`. Создаваемый ею итератор будет возвращать те значения из `iterable`, для которых функция `func` возвращает `True`:

In [77]:
f = filter(str.isdigit, ['1', 'one', '2'])
list(f)

['1', '2']

In [78]:
print(list(filter(lambda x: x > 0, range(-5, 5))), end='\n\n')

print('чтобы понять, какие значения возвращал итератор filter:')

print(*zip(range(-5, 5), 
           map(lambda x: x > 0, range(-5, 5))),
      sep='\n')

[1, 2, 3, 4]

чтобы понять, какие значения возвращал итератор filter:
(-5, False)
(-4, False)
(-3, False)
(-2, False)
(-1, False)
(0, False)
(1, True)
(2, True)
(3, True)
(4, True)


Убедимся, что каждая из использованных нами функций возвращает итераторы:

In [79]:
for obj in [m, z, e, f]:
    print(obj)
    print(type(obj))
    print(isinstance(obj, Iterator))

<class 'map'>
True
<class 'zip'>
True
<class 'enumerate'>
True
<class 'filter'>
True


### functools

С этим модулем мы еще встретимся в будущем

Встроенный модуль `functools` хранит в себе функции более высокого порядка: функции, которые воздействуют на другие функции или возвращают их. В общем, любой вызываемый объект может рассматриваться как функция для целей данного модуля.

https://docs.python.org/3.11/library/functools.html

#### functools.reduce

In [80]:
from functools import reduce

In [81]:
sequence = [1, 2, 3, 4, 5]

print(reduce(lambda x, y: x * y, sequence[:2]))  # [1, 2]
print(reduce(lambda x, y: x * y, sequence[:3]))  # [1, 2, 3]
print(reduce(lambda x, y: x * y, sequence[:4]))  # [1, 2, 3, 4]
print(reduce(lambda x, y: x * y, sequence))      # [1, 2, 3, 4, 5]

2
6
24
120


In [82]:
reduce(lambda x, y: x + y, [1, 2, 3, 4])

10

#### functools.partial

`partial(func, /, *args, **keywords)` - создает новую функцию на основе `func`, "заморозив" ей некоторые аргументы. Например, если `func` принимает два аргумента func(arg1, arg2), то вызов `partial(func, arg2=10)` вернет функцию func с замороженным аргументом `arg2=10`

In [84]:
from functools import partial

In [87]:
# The partial() is used for partial function application which “freezes” some portion of 
# a function’s arguments and/or keywords resulting in a new object.

basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

18

In [90]:
type(basetwo)

functools.partial

In [85]:
int('10010')

10010

In [86]:
int('10010', base=2)

18

### itertools

[docs: Functions creating iterators for efficient looping](https://docs.python.org/3.11/library/itertools.html#module-itertools)

Встроенный модуль `itertools` содержит ряд часто используемых итераторов, а также функции для объединения нескольких итераторов. Вместе они образуют “итераторную алгебру”, позволяющую лаконично и эффективно создавать специализированные инструменты на чистом Python.

#### Бесконечные генераторы

In [91]:
from itertools import count, cycle, repeat

`count(start=0, step=1)` - бесконечный генератор чисел со `start` с шагом `step`

In [92]:
for x in count():
    if x > 2: 
        break
    else:
        print(x)

print('----------')

for x in count(-5, 3):
    if x > 2: 
        break
    else:
        print(x)

0
1
2
----------
-5
-2
1


`cycle(iterable, /)` - возвращает по одному элементы из `iterable`. Добравшись до конца, начинает сначала:

In [93]:
i = 0
for x in cycle([1, 2, 3]):
    if i > 4: 
        break
    else:
        print(x)
    i += 1

1
2
3
1
2


`repeat(object [,times])` - возвращает `object` бесконечное число раз, либо заданное в необязательном параметре `times`

In [94]:
for elem in repeat('AAAA', 3):
    print(elem)

AAAA
AAAA
AAAA


Вызовем все эти бесконечные итераторы вместе с помощью `zip`:

In [96]:
for elems in zip(count(), 
                 cycle([1, 2, 3]), 
                 repeat("GG"), 
                 range(7)):
    print(elems)

(0, 1, 'GG', 0)
(1, 2, 'GG', 1)
(2, 3, 'GG', 2)
(3, 1, 'GG', 3)
(4, 2, 'GG', 4)
(5, 3, 'GG', 5)
(6, 1, 'GG', 6)


#### Комбинаторика

Есть итераторы, возвращающие:
- декартово произведение
- перестановки
- сочетания
- сочетания с повторениями

In [97]:
from itertools import product, permutations, combinations
from itertools import combinations_with_replacement

`itertools.product` - возвращает декартово произведение последовательностей. Можно красиво переписывать вложенные циклы:

In [98]:
for i in ['a', 'b', 'c']:
    for j in range(1, 4):
        print(i, j)

print()

#Эквивалентно:
for i, j in product(['a', 'b', 'c'], range(1, 4)):
    print(i, j)

a 1
a 2
a 3
b 1
b 2
b 3
c 1
c 2
c 3

a 1
a 2
a 3
b 1
b 2
b 3
c 1
c 2
c 3


`itertools.permutations(iterable, r=None)` - всевозможные перестановки r элементов из `iterable`

In [99]:
list(permutations(['A', 0, True], r=2))

[('A', 0), ('A', True), (0, 'A'), (0, True), (True, 'A'), (True, 0)]

Если `r == None`, то берутся все элементы:

In [100]:
list(permutations(['A', 0, True]))

[('A', 0, True),
 ('A', True, 0),
 (0, 'A', True),
 (0, True, 'A'),
 (True, 'A', 0),
 (True, 0, 'A')]

`itertools.combinations(iterable, r)` - всевозможные сочетания r элементов

In [101]:
list(combinations([1, 2, 3, 4, 5], r=3))

[(1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5)]

`itertools.combinations_with_replacement(iterable, r)` - всевозможные сочетания r элементов с повторениями

In [102]:
list(combinations_with_replacement(['banana', 'apple', 'pear'], 2))

[('banana', 'banana'),
 ('banana', 'apple'),
 ('banana', 'pear'),
 ('apple', 'apple'),
 ('apple', 'pear'),
 ('pear', 'pear')]

В этом модуле хранится еще много других прекрасных итераторов, с которыми можно быстро ознакомиться в таблице наверху [страницы](https://docs.python.org/3/library/itertools.html). Например, `accumulate`.

In [103]:
from itertools import accumulate

list(accumulate([1, 2, 3, 10, 20, 100]))

[1, 3, 6, 16, 36, 136]

## Namespaces

Пространство имён -- маппинг из имен переменных в объекты.

**locals()** - возвращает текущий namespace в виде словаря <br>
**globals()** - возвращает namespace модуля

In [111]:
# dir()

['D',
 'Generator',
 'Hashable',
 'In',
 'Iterable',
 'Iterator',
 'Out',
 '_',
 '_10',
 '_100',
 '_101',
 '_102',
 '_103',
 '_104',
 '_105',
 '_106',
 '_108',
 '_12',
 '_14',
 '_17',
 '_18',
 '_19',
 '_3',
 '_31',
 '_33',
 '_34',
 '_4',
 '_42',
 '_49',
 '_5',
 '_51',
 '_52',
 '_54',
 '_57',
 '_61',
 '_63',
 '_66',
 '_67',
 '_68',
 '_69',
 '_7',
 '_70',
 '_71',
 '_74',
 '_77',
 '_8',
 '_82',
 '_85',
 '_86',
 '_87',
 '_88',
 '_89',
 '_9',
 '_90',
 '_99',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i10',
 '_i100',
 '_i101',
 '_i102',
 '_i103',
 '_i104',
 '_i105',
 '_i106',
 '_i107',
 '_i108',
 '_i109',
 '_i11',
 '_i110',
 '_i111',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',

In [2]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'locals()', 'globals()'],
 '_oh': {1: {...}},
 '_dh': [WindowsPath('D:/Desktop/AImasters/питон/lecture06/lecture06')],
 'In': ['', 'locals()', 'globals()'],
 'Out': {1: {...}},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001A9088EF2E0>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x1a9088efeb0>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x1a9088efeb0>,
 'open': <function io.open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)>,
 '_': {...},
 '__': '',
 '___': '',
 '_i': 'locals()',
 '_ii': '',
 '_iii': '',
 '_i1': 'locals()',
 '_1': {...},
 '_i2': 'globals()'}

In [1]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'locals()'],
 '_oh': {},
 '_dh': [WindowsPath('D:/Desktop/AImasters/питон/lecture06/lecture06')],
 'In': ['', 'locals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001A9088EF2E0>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x1a9088efeb0>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x1a9088efeb0>,
 'open': <function io.open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)>,
 '_': '',
 '__': '',
 '___': '',
 '_i': '',
 '_ii': '',
 '_iii': '',
 '_i1': 'locals()'}

In [105]:
locals() is globals()

True

In [3]:
value = 42
print(globals()['value'])

globals()['value'] = 100500
print(value)

42
100500


#### Циклы и условия не создают своё пространство имён

In [115]:
if True:
    value_assigned_in_if = 1
    
for loop_counter in range(10):
    print(f'{loop_counter = }')
    value_assigned_in_for = 2
    
print(f'{loop_counter = } after cycle')
print(f'{value_assigned_in_if = }')
print(f'{value_assigned_in_for = }')

loop_counter = 0
loop_counter = 1
loop_counter = 2
loop_counter = 3
loop_counter = 4
loop_counter = 5
loop_counter = 6
loop_counter = 7
loop_counter = 8
loop_counter = 9
loop_counter = 9 after cycle
value_assigned_in_if = 1
value_assigned_in_for = 2


#### Функции создают своё пространство имён

In [116]:
value = 0

def foo():
    value = 1
    print(f'{value = }')
    
    print('locals:', locals()['value'])
    print('globals:', globals()['value'])
    
    
foo()
print(f'{value = }')

value = 1
locals: 1
globals: 0
value = 0


### Область видимости (scope)

Правило LEGB:
1. Local - имена, определенные внутри функции (и не помеченные global)
2. Enclosing-function locals - имена в области видимости всех оборачивающих (enclosing) функций, в порядке уменьшения глубины
3. Global - имена, определенные на уровне модуля или посредством global
4. Built-in - предопределенные (range, enumerate, zip, map, open, ...)

In [118]:
value = 111

def foo():
    value = 222
    
    def bar():
        value = 333
        print('bar local scope', value)
    
    bar()
    print('enclosing scope value', value)
    
foo()
print('global value', value)

bar local scope 333
enclosing scope value 222
global value 111


In [117]:
bar()

NameError: name 'bar' is not defined

#### Пример LEGB

In [119]:
range

range

In [120]:
print(range)

<class 'range'>


In [121]:
def foo():
    def bar():
        print('built-in:', range)
    bar()
foo()

built-in: <class 'range'>


In [122]:
range = 'global range'

def foo():
    def bar():
        print('global:', range)
    bar()
foo()

global: global range


In [123]:
range = 'global range'

def foo():
    range = 'enclosing-function range'
    def bar():
        print('enclosing-function:', range)
    bar()
foo()

enclosing-function: enclosing-function range


In [124]:
range = 'global range'

def foo():
    range = 'enclosing-function range'
    def bar():
        range = 'local range'
        print(locals())
        print('local:', range)
    bar()
foo()

{'range': 'local range'}
local: local range


In [125]:
range(1)

TypeError: 'str' object is not callable

In [126]:
globals()['range']

'global range'

In [127]:
'global range'(1)

<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
/var/folders/v8/2cskhqjj0_g0zp3824glf1tm0000gn/T/ipykernel_47796/4101326813.py:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  'global range'(1)


TypeError: 'str' object is not callable

In [128]:
del range

range(10)

range(0, 10)

### Ключевое слово global

In [ ]:
value = 1

def foo():
    value = 2
    
    def bar():
        global value
        value = 3
    
    bar()
    print('enclosing scope value', value)
    
foo()
print('global value', value)

### Ключевое слово nonlocal

In [ ]:
value = 1

def foo():
    value = 2
    
    def bar():
        nonlocal value
        value = 3
    
    bar()
    print('enclosing scope value', value)
    
foo()
print('global value', value)

Пространства имён в python **статические** <br>
Определение любого используемого в коде обьекта можно найти без запуска программы.

In [ ]:
value = 1

def foo():
    
    print(value)
    
    def bar():
        print(value)
    
    bar()
    value = 2
    
foo()